In [1]:
import pandas as pd

df = pd.read_csv('data_dac.csv', sep = ',', header = 0, error_bad_lines=False, date_parser=['posted_at'])
df = df[df.named_id.notnull() == True ]
df.nb_follower = df.nb_follower.astype('int32')
df.nb_following = df.nb_following.astype('int32')
df.length = df.length.astype('int32')
df.orthographe = df.orthographe.astype('float64')
df.reset_index(inplace = True)

b'Skipping line 75: expected 17 fields, saw 18\nSkipping line 106: expected 17 fields, saw 18\nSkipping line 138: expected 17 fields, saw 18\nSkipping line 165: expected 17 fields, saw 18\nSkipping line 166: expected 17 fields, saw 19\nSkipping line 187: expected 17 fields, saw 19\nSkipping line 261: expected 17 fields, saw 18\nSkipping line 269: expected 17 fields, saw 19\nSkipping line 339: expected 17 fields, saw 19\nSkipping line 702: expected 17 fields, saw 19\nSkipping line 731: expected 17 fields, saw 18\nSkipping line 773: expected 17 fields, saw 18\nSkipping line 844: expected 17 fields, saw 19\nSkipping line 1262: expected 17 fields, saw 18\nSkipping line 1293: expected 17 fields, saw 18\nSkipping line 1323: expected 17 fields, saw 20\nSkipping line 1577: expected 17 fields, saw 18\nSkipping line 1685: expected 17 fields, saw 19\nSkipping line 1967: expected 17 fields, saw 18\nSkipping line 1973: expected 17 fields, saw 18\nSkipping line 2102: expected 17 fields, saw 18\nSkip

Suite à l'analyse des features que nous avons effectué dans 'pretraitement' nou allons maintenant procéder à des analyses sur les textes des tweets pour voir si des tendances se dégagent et ce que l'on peut en tirer.

Nous allons commencer par définir une fonction permettant de tokenizer les tweets que nous souhaitons analyser. Nous allons d'abord supprimer tout les hyperliens et mention d'utilisateur qui risque de perturber notre analyse. Nous allons aussi supprimer les stopwords et découper les hastags en fonction des majuscules (#GiletJaune devient 'Gilet Jaune')

In [2]:
import re
import fr_core_news_md
nlp = fr_core_news_md.load()

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from Emojilist import emojilist

In [7]:


stopwords_fr = stopwords.words('french')


def tokenize(message):
        message = re.sub(r'(?:\@|https?\://)\S+', '', message)  # remove links and @username
        message = re.sub(r'[^\w\s]', ' ', message)  # remove non-alphanumeric characters
        doc = nlp (message)
        stopWords = new_stopwords
        hashtag = False
        list = [str(token) for token in doc]
        tokens = []
        for elt in list :
            #remove stopwords
            if elt not in stopWords :
                #couper les lettres qui se repètent plus de 2fois de suite
                for i in range(len(elt)-1,1, -1):
                    if elt[i] == elt [i-1] and elt[i] == elt [i-2] :
                        elt = elt[:i]+elt[i+1:]
                #On coupe le mot si il s'agit du terme après un hastag
                if hashtag == True :
                    decoup = re.findall('[A-Z][^A-Z]*',elt)
                    for word in decoup :
                        tokens.append(word)
                    hashtag = False
                elif elt == '#' :
                    hashtag = True
                elif elt in emojilist :
                    pass
                else :
                    tokens.append(elt)
        return tokens

In [4]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import collections



def cluster_texts_Kmeans( texts, k=5):
        """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
        vectorizer = TfidfVectorizer(
            tokenizer=tokenize,
            max_features=1500,
            max_df=0.7,
            min_df=5,
            lowercase=True)

        tfidf_model = vectorizer.fit_transform(texts)
        km_model = KMeans(n_clusters=k, init='k-means++', n_init=10)
        km_model.fit_predict(tfidf_model)
        
        clustering = collections.defaultdict(list)

        print("Top terms per cluster:")
        order_centroids = km_model.cluster_centers_.argsort()[:, ::-1]
        #print(order_centroids)
        terms = vectorizer.get_feature_names()
        for i in range(k):
            print("\n Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])


        

In [5]:
print(cluster_texts_Kmeans(df.text))

Top terms per cluster:

 Cluster 0:
 a
   
 les
 fait
 tout
 plus
 faire
 si
 bien
 rien

 Cluster 1:
   
 plus
 bien
 merci
 fait
 

 si
 quand
 trop
 non

 Cluster 2:
   
 les
 trop
 vrai
 via
 ah
 dit
 merci
 live
 père

 Cluster 3:
 ça
   
 si
 fait
 a
 les
 comme
 bien
 plus
 va

 Cluster 4:
 les
   
 ils
 tous
 gens
 plus
 

 là
 comme
 quand
None


Nous remarquons qu'il reste beaucoup de stopwords dans les mots les plus fréquents. Nous allons donc compléter la liste de stopwords pour qu'elle soit plus complète que la liste de base présente dans ntlk et modifier la fonction tokenize pour qu'elle prenne cette nouvelle liste

In [6]:
new_stopwords = stopwords.words('french') + ['oui','non','merci','a','bah','toujours','va','dit','alors','juste','sujet','au','la','sur','aucuns','le','ta','aussi','les','tandis',
'autre','leur','tellement','avant','là','tels','avec','ma','tes','avoir','maintenant','ton','bon','mais','tous',
'car','mes','tout','ce','mine','trop','cela','moins','très','ces','mon','tu','ceux','mot','voient','chaque','même','vont',
'ci','ni','votre','comme','nommés','vous','comment','notre','vu','dans','nous','ça','des','ou','étaient','du','où','état',
'dedans','par','étions','dehors','parce','été','depuis','pas','être','devrait','peut','doit','peu','donc','plupart',
'dos','pour','début','pourquoi','elle','quand','elles','que','en','quel','encore','quelle','essai','quelles','est','quels',
'et','qui','eu','sa','fait','sans','faites','ses','fois','seulement','font','si','hors','sien','ici','son','il','sont',
'ils','sous','je','soyez','plus']


In [8]:
print(cluster_texts_Kmeans(df.text))

Top terms per cluster:

 Cluster 0:
   
 3
 via
 ah
 jamais
 no
 malaise
 live
 père
 pareil

 Cluster 1:
   
 

 quoi
 cette
 mdrr
 faut
 2
 mdr
 jamais
 ah

 Cluster 2:
 bien
   
 sûr
 sais
 toutes
 deux
  

 ah
 connu
 aime

 Cluster 3:
 france
   
 honte
 entre
 années
 temps
 5
 
  
 3
 bientôt

 Cluster 4:
 faire
   
 vais
 faut
 plaisir
 rien
 envie
 partie
 2019
 aller
None


Maintenant on obtient des termes plus pertinents pour qualifier les clusters. Néanmoins il nous reste à définir le nombre de cluster qu'il serait pertinent de faire car pour l'instant il est fixé par nous avec la méthode des K_means.

Pour cela nous avons regardé les différents algorithme qui permettait de définir automatiquement le nombre de cluster basé sur la densité et nous avons choisi de nous interesser à la fonction DBSCAN

In [9]:
from sklearn.cluster import DBSCAN

def cluster_texts_DBSCAN( texts):
        vectorizer = TfidfVectorizer(
            tokenizer=tokenize,
            max_features=1500,
            max_df=0.7,
            min_df=5,
            lowercase=True)

        tfidf_model = vectorizer.fit_transform(texts)
        db = DBSCAN(eps=0.5, min_samples=5)
        db.fit_predict(tfidf_model)
        labels = db.labels_
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        print('Estimated number of clusters: %d' % n_clusters_)
        print('Estimated number of noise points: %d' % n_noise_)
        for j in range (n_clusters_) :
            print("\n Cluster %d:" % j)
            for i in range(len(labels)) :
                if labels[i] == j :
                    print('tweet : ' + str(df.text[i]))



In [10]:
print(cluster_texts_DBSCAN(df.text))

Estimated number of clusters: 30
Estimated number of noise points: 2607

 Cluster 0:
tweet : Rosier muscat du chili. Maintenant  l'applicateur fonctionne.
tweet : @cinnamonfraize à ce qu'il paraît, c'est ton Fraizanniversaire. Bisouuuuus😙
tweet : @W9 Merci
tweet : Boffum
tweet : @JobCenter91448 Ja moin
tweet : @WeAreMarmalade Audi
tweet : L’Humanité essentielle au pluralisme !
tweet : sur la gâchette j’ai parkinson
tweet : Depoparfum https://t.co/xW7AseimVN
tweet : Signalez @GauldenBoy s’il vous plaît il usurpe mon identité
tweet : Définition du notion du TIMING :
tweet : @relajandome camaron bombay
tweet : @Yanfilipe99 cel n quer carregar mn, estragou
tweet : même moi
tweet : Remarkable ignorance. https://t.co/ihFHzNONBX
tweet : @Hadieesta Réal Madrid😍
tweet : @bakIIava Un te anglishtes
tweet : @JabraFrance Jabra tu es mon essentiel @Chath35311979
tweet : Des structures à la rêcherche d’une squad
tweet : @rubixdolan Moist
tweet : Natalie Gauvreau https://t.co/MRwE8xE3fW
tweet : Merci 

tweet : @DaylingAguilar Déjame ser 👻
tweet : Donghae es lo mas angst de la vida😭
tweet : @lbyleem Je suis chirurgienne.
tweet : @styledlov bah on retournera à La Rochelle alors 😭
tweet : @CapelleLili Mais carrément 🤤
tweet : @Lustfullman75 Je n'étais pas de mauvaise humeur, quand je suis de mauvaise humeur je ne répond même pas quand on m… https://t.co/N92Srf3Sqy
tweet : Tfais trop lmec
tweet : 4v1 je suis encore traumatisé https://t.co/Iz0P5pv3zh
tweet : @cjstoney9 Ridiculous 😂
tweet : J'ai cassé une télé sur le but de  Mbappe
tweet : On en est donc là.
tweet : @PaixQuen Merciii
tweet : @tiarakion Lemme come
tweet : 2c2d74a3-f9e5-4d0a-a4b3-37a09a5879ce
tweet : @Gaston_Botez N'empêche elle a des grosses couilles
tweet : portrait❤ https://t.co/5cSskdANd9
tweet : 2c2d74a3-f9e5-4d0a-a4b3-37a09a5879ce
tweet : @NatmoLick Bon appétit
tweet : Excellente méthode.
tweet : @ITSLEOTISTE jcroyais tu vnais à rennes?
tweet : Fabulous https://t.co/eF4zjD9hag
tweet : 144bcb4f-ee70-40fd-a2ce-c8d7ef3bf4

tweet : @laviecestduvide @Tournad96303389 @VraiGaulliste @Pale_rider796 @gregoryroose @MarcEdgar75 @gwendoSta @LeGwened9… https://t.co/XXPsZVG47f
tweet : @Figaro_Live @Le_Figaro Les exemples c’est seulement quand cela arrange les puissants
tweet : La rédaction d'Enviscope est sur #BePOSITIVE2019 ! https://t.co/KFAIPP47dK
tweet : @Tournad96303389 @VraiGaulliste @Pale_rider796 @gregoryroose @MarcEdgar75 @gwendoSta @LeGwened9 @SJallamion… https://t.co/TfEjSxnMR9
tweet : @RM_ied 😂😂😂😂depuis lors c'est compliqué
tweet : @Lewiis44 @EvilawFTN @ZENOOBIE @B4zing4_2 @Gotaga @catsswhites t qUi ? https://t.co/W3fNu0hezr
tweet : Et maintenant la contraception obligatoire… en attendant l’avortement forcé ? https://t.co/tUPr7r87hv
tweet : ez money waldas pag siena fair jusq 🤦🏻‍♀️
tweet : Vias: dix recommandations de sécurité routière pour sauver plus de vies - Tinlot https://t.co/VdIFdXUgAS
tweet : @Nabil_TW @Malika_PSG T'es sévère avec Verratti! Il a été énorme aussi!
tweet : Le Congrès enterre le « 

tweet : https://t.co/fRublmbCRg on @EmmanuelMacron  t'as vécus
tweet : @ stop homofobie a été suspendu. https://t.co/B8QfGLmBen
tweet : @AminataDasylva @Macky_Sall Votons la continuité
tweet : Elections dans les chambres d’agriculture : les partisans des pesticides toujours majoritaires… https://t.co/KUZwcmThlB
tweet : @ThomasRbrt @noemieschulz Non, la passerelle n'était pas sur l'itinéraire déclaré.
tweet : #CANU20 - Mamoutou Kané vise un troisième mondial et rêve du trophée de la CAN https://t.co/OH8cZbigLg https://t.co/glSiFHnBQR
tweet : @Challenges Ont est avec vous les belges ! 😘
tweet : @plongedehors on est corda 🤲🏾
tweet : @rebixkm @ad135_ Jss dépasser 😭
tweet : Les fichiers ne portent que sur les condamnations et non sur les soupçons déplorent les dénonciateurs.  Comme si un… https://t.co/OZSHlJGjYn
tweet : @Mimib13100 @stefdumas @Girondinsjohn @Loup_Augier @saelno @emo9ParisRP @LaFrancematuer @_LICRA_ @FPotier_Dilcrah… https://t.co/VcXylW8GUR
tweet : DIAURA - Ms.Psycho https:/

tweet : la végétation....les fruits.....a bitch is missed
tweet : @JvyTrillv @ZametriaSimonee Ridiculous
tweet : @rozzleberry @OwenJones84 @MikeSegalov Author of dangerous hero
tweet : Umtiti dans les couloirs ❤❤
tweet : @JeanLuc92i @pxrbt @humbertrv @ParaCLET777888 Merci captain obvious donc pas dnenfer, pas de paradis , pas de lumbe… https://t.co/UiVhyFeLM2
tweet : @EA_FIFA_France @SerieA @spfl Mais où il sont c dce ?

 Cluster 3:
tweet : @paulo_krm Ducoup je rt
tweet : solenn_le_goff: RT claire_kilic: RT PleziApp: [Interview] Après sa conférence à IMF, randfish a répondu aux questio… https://t.co/KEwuHgOhA8
tweet : RT CorpRanger #JFCAvrilLavigne
tweet : @InconnuBras ton rt rempli de jalousie. Franchement c'est moche.
tweet : RT test5f1798: que? : 38ab9a93-09e6-4bd3-9a35-1f9c37af102d
tweet : RT reus_cat: https://t.co/wMnH3UJ048 #Reus #Informatico

 Cluster 4:
tweet : @Meriouma016 Bonjour Meriem 😊👋
tweet : @bfm_eco @BFMTV Bonjour la qualité des soins. Pour les pzuvres: Médecine privée

tweet : @LouiseouVictor @mx_glitter Bon aller retourne fantasme sur des trucs qui arriveront que dans des fanfictions ça se… https://t.co/u52URkhqzO
tweet : @svuvvge ALLER TAGEUUULE

 Cluster 20:
tweet : @carterforva VoTe MaNiPuLaTiOn
tweet : VOTE LISANATION!!!
tweet : Au #Guatemala, Otto Pérez, accusé de corruption comme @moisejovenel est privé de son immunité par un vote au Parlem… https://t.co/WTq1yTxAA8
tweet : #BRITVIDLITTLEMIX . Vote https://t.co/v6oGOueYko
tweet : vote ranch

 Cluster 21:
tweet : mds quero
tweet : kkkkkkkkkkkk mds
tweet : Mds meu
tweet : @meucarotuitter Kkkkkkkkkkk mds
tweet : MDS MDS MDS MDS MDS MDS https://t.co/2PomG0pEvs

 Cluster 22:
tweet : @nateklain C’est vrai
tweet : @Mln_Lbn @VitalePv franchement c'est vrai ...💋🌹🥰
tweet : @imnotalone tu t'appelles pas Emma Tseule en vrai ? Moi je croyais !
tweet : Le volant attire les groupies c’est vrai ☝🏾🤷🏾‍♂️
tweet : @Vin_cia_noeud Oui c’est vrai
tweet : @fourmichto tellement pas en vrai :(

 Cluster 23:
tweet : @bet

L'analyse des clusters réalisé avec DBSCAN nous montre que beaucoup de point (75%) sont classés comme bruit et seul 25% sont clusterisés avec les paramètres actuels. 
Si nous voulons que plus de points soient clusterisés il faudrait que nous diminuions le paramètres eps pour DBSCAN qui catégorise a partir de quelle distance on considère 2 points comme proches. Néanmoins si l'on fait ça alors la pertinence de nos clusters risque de diminuer alors qu'elle est déjà faible pour le cluster 0 (qui n'est pourtant pas le cluster poubelle).

C'est pourquoi il faudrait mener des tests avec plusieurs valeurs de 'eps' et 'min_samples' pour voir quelle combinaison nous donne des clusters pertinents pour l'analyse du dataset.